In [1]:
from topologicpy.Graph import Graph
from topologicpy.Helper import Helper
import topologicpy
from topologicpy.Topology import Topology
from topologicpy.Vertex import Vertex
from topologicpy.Wire import Wire
from topologicpy.Face import Face
from topologicpy.Shell import Shell
from topologicpy.Cell import Cell
from topologicpy.Dictionary import Dictionary
from topologicpy.Cluster import Cluster

print(Helper.Version())
renderer = "browser"

The version that you are using (0.8.15) is the latest version available on PyPI.


In [2]:
print(topologicpy.__file__)  # Zeigt den Pfad der geladenen TopologicPy-Version

/Users/roger/miniconda3/lib/python3.12/site-packages/topologicpy/__init__.py


# Cube Grid Function

In [3]:
def create_cubes_and_shell():
    """
    Creates a 2 x 3 grid of 3×3×3 cubes stacked 3 levels high,
    separated by 0.1 m, and an enclosing bounding shell 
    expanded by 0.3 m in all directions.

    Returns a dict with keys:
      - "cubes": a cluster of all cubes
      - "boundingShell": a thick shell offset by 0.3 m
      - "allGeometry": combined geometry as a single cluster
    """

    # 1. Basic parameters
    cell_size = 3.0  # 3m cubes
    gap = 0.1  # 10cm gap
    num_x = 2
    num_y = 3
    num_z = 3
    offset_shell = 0.3  # 30 cm offset for shell

    # 2. Generate all cubes
    cubes_list = []
    for ix in range(num_x):
        for iy in range(num_y):
            for iz in range(num_z):
                # Offsets for the current cube
                offset_x = ix * (cell_size + gap)
                offset_y = iy * (cell_size + gap)
                offset_z = iz * (cell_size + gap)

                # Use Cell.Prism to make each 3×3×3 cube
                # at origin=(0,0,0), but "lowerleft" so 
                # (0,0,0) is the lower-left corner
                base_cube = Cell.Prism(
                    origin=Vertex.Origin(),
                    width=cell_size,
                    length=cell_size,
                    height=cell_size,
                    placement="lowerleft"  # ensures bottom face is at z=0
                )

                # Move the cube into position
                moved_cube = Topology.Translate(
                    base_cube,
                    offset_x, offset_y, offset_z
                )

                # Attach some dictionary info to the cube
                d = Dictionary.ByKeysValues(
                    ["ix", "iy", "iz"],
                    [ix, iy, iz]
                )
                moved_cube = Topology.SetDictionary(moved_cube, d)

                cubes_list.append(moved_cube)

    # Combine into a single cluster
    cubes_cluster = Cluster.ByTopologies(cubes_list)

    # 3. Compute bounding coords (min_x, max_x, etc.) for the cubes
    all_verts = Topology.Vertices(cubes_cluster)
    x_vals = [Vertex.X(v) for v in all_verts]
    y_vals = [Vertex.Y(v) for v in all_verts]
    z_vals = [Vertex.Z(v) for v in all_verts]

    min_x, max_x = min(x_vals), max(x_vals)
    min_y, max_y = min(y_vals), max(y_vals)
    min_z, max_z = min(z_vals), max(z_vals)

    # 4. Build the "inner" bounding prism (exactly around cubes)
    inner_prism = Cell.Prism(
        origin=Vertex.ByCoordinates(min_x, min_y, min_z),
        width=(max_x - min_x),
        length=(max_y - min_y),
        height=(max_z - min_z),
        placement="lowerleft"  # means that (min_x, min_y, min_z) 
        # is the lower-left corner
    )

    # 5. Build the "outer" bounding prism = expanded by 0.3 m
    outer_prism = Cell.Prism(
        origin=Vertex.ByCoordinates(min_x - offset_shell,
                                    min_y - offset_shell,
                                    min_z - offset_shell),
        width=(max_x - min_x) + 2 * offset_shell,
        length=(max_y - min_y) + 2 * offset_shell,
        height=(max_z - min_z) + 2 * offset_shell,
        placement="lowerleft"
    )

    # 6. Subtract the inner from outer => thick bounding shell
    bounding_shell = Topology.Difference(outer_prism, inner_prism)

    # 7. Combine everything
    final_cluster = Cluster.ByTopologies([cubes_cluster, bounding_shell])

    return {
        "cubes": cubes_cluster,
        "boundingShell": bounding_shell,
        "allGeometry": final_cluster
    }

# Create Cubes Cluster 2*3 Grid

In [4]:
result = create_cubes_and_shell()

cubes = result["cubes"]
shell = result["boundingShell"]
all_geo = result["allGeometry"]

In [5]:
Topology.Show(cubes, shell, renderer=renderer)

# Face Grid Function

In [22]:
import topologic_core as topologic

def rectangle_face(min_x, min_y, max_x, max_y):
    """
    Manually build a rectangle face in the XY plane from (min_x, min_y) to (max_x, max_y).
    1. Make four vertices
    2. Make four edges
    3. Combine into a wire
    4. Make a face by Face.ByExternalBoundary
    """
    # 1. Corner vertices in 2D (z=0)
    vA = topologic.Vertex.ByCoordinates(min_x, min_y, 0)
    vB = topologic.Vertex.ByCoordinates(max_x, min_y, 0)
    vC = topologic.Vertex.ByCoordinates(max_x, max_y, 0)
    vD = topologic.Vertex.ByCoordinates(min_x, max_y, 0)

    # 2. Four edges
    eAB = topologic.Edge.ByStartVertexEndVertex(vA, vB)
    eBC = topologic.Edge.ByStartVertexEndVertex(vB, vC)
    eCD = topologic.Edge.ByStartVertexEndVertex(vC, vD)
    eDA = topologic.Edge.ByStartVertexEndVertex(vD, vA)

    # 3. Single wire from these edges
    rect_wire = topologic.Wire.ByEdges([eAB, eBC, eCD, eDA], False)

    # 4. Face from the wire
    rect_face = topologic.Face.ByExternalBoundary(rect_wire, False)
    return rect_face

def create_faces_and_shell():
    """
    Creates a single-floor 2 x 3 grid of 3×3 square faces (XY-plane),
    separated by 0.1 m, plus an enclosing bounding ring 0.3 m offset.
    
    Returns dict with keys:
      - "faces": a *LIST* of the 6 square faces
      - "boundingShell": a Face ring offset by 0.3 m
      - "allGeometry": everything as a single Cluster
    """

    cell_size = 3.0  # 3 m squares
    gap = 0.1        # 0.1 m gap in x,y
    num_x = 2
    num_y = 3
    offset_shell = 0.3  # extra boundary offset

    # 1. Build 6 squares in a 2×3 layout
    face_list = []
    for ix in range(num_x):
        for iy in range(num_y):
            # Compute lower-left corner of this square
            x0 = ix * (cell_size + gap)
            y0 = iy * (cell_size + gap)
            # Make a 3×3 face
            f = rectangle_face(x0, y0, x0 + cell_size, y0 + cell_size)

            # Attach a dictionary if you want
            d = topologic.Dictionary.ByKeysValues(
                ["ix", "iy"],
                [topologic.StringAttribute(str(ix)), topologic.StringAttribute(str(iy))]
            )
            f.SetDictionary(d)

            face_list.append(f)

    # 2. Gather all face vertices to find bounding box
    #    (We won't combine faces into a single cluster here, so we do it manually)
    all_verts = []
    for face in face_list:
        these_verts = []
        face.Vertices(None, these_verts)
        all_verts.extend(these_verts)

    xs = [v.X() for v in all_verts]
    ys = [v.Y() for v in all_verts]

    min_x, max_x = min(xs), max(xs)
    min_y, max_y = min(ys), max(ys)

    # 3. Create 'inner' rectangle exactly bounding the squares
    inner_rect = rectangle_face(min_x, min_y, max_x, max_y)

    # 4. Create 'outer' rectangle = expanded by 0.3 m
    out_min_x = min_x - offset_shell
    out_min_y = min_y - offset_shell
    out_max_x = max_x + offset_shell
    out_max_y = max_y + offset_shell
    outer_rect = rectangle_face(out_min_x, out_min_y, out_max_x, out_max_y)

    # 5. Difference => ring bounding shell
    bounding_shell = topologic.Topology.Difference(outer_rect, inner_rect)

    # 6. Combine everything into one cluster for "allGeometry"
    #    This cluster has the bounding shell face plus the 6 separate faces
    all_geo = topologic.Cluster.ByTopologies(face_list + [bounding_shell], False)

    return {
        "faces": face_list,              # <-- a plain Python list of faces
        "boundingShell": bounding_shell, # the ring offset by 0.3
        "allGeometry": all_geo           # cluster of everything
    }

# Compute 2*3 Face Grid with Shell

In [23]:
result = create_faces_and_shell()
faces = result["faces"]         
shell = result["boundingShell"]  
geo = result["allGeometry"]      

In [24]:
Topology.Show(shell, faces, renderer=renderer)

In [9]:
print("Shell: " + shell.__str__())
print("Faces: " + faces.__str__())

Shell: <topologic_core.Face object at 0x128a60170>
Faces: [<topologic_core.Face object at 0x128a380f0>, <topologic_core.Face object at 0x128a3a4f0>, <topologic_core.Face object at 0x128a3a7f0>, <topologic_core.Face object at 0x128a380b0>, <topologic_core.Face object at 0x128a39fb0>, <topologic_core.Face object at 0x128a38fb0>]


# Compute Skeleton

In [10]:
class Shell2():
    @staticmethod
    def ByDisjointFaces(externalBoundary,
                        faces,
                        maximumGap: float = 0.5,
                        mergeJunctions: bool = False,
                        threshold: float = 0.5,
                        uSides: int = 1,
                        vSides: int = 1,
                        transferDictionaries: bool = False,
                        mantissa: int = 6,
                        tolerance: float = 0.0001):
        """
        Creates a shell from an input list of disjoint faces. THIS IS STILL EXPERIMENTAL
        """
        from tqdm.auto import tqdm
        from topologicpy.Vertex import Vertex
        from topologicpy.Edge import Edge
        from topologicpy.Wire import Wire
        from topologicpy.Face import Face
        from topologicpy.Cluster import Cluster
        from topologicpy.Helper import Helper
        from topologicpy.Topology import Topology
        from topologicpy.Grid import Grid
        from topologicpy.Dictionary import Dictionary

        # ---------------------- HELPER FUNCTIONS ---------------------- #
        def force_list_of_edges(obj):
            """
            Convert 'obj' to a list of Edge objects, if possible.
            obj might be:
              - a single Edge
              - a Wire
              - a Cluster
              - a list of edges
              - or something else
            """
            if isinstance(obj, list):
                # Already a list => assume these are edges or empty
                return obj
            # Otherwise, attempt to gather edges from it
            return Topology.Edges(obj)

        def removeShards(edges_or_wire, hostTopology, maximumGap=0.5):
            # Ensure 'edges' is a python list of edges
            edges_list = force_list_of_edges(edges_or_wire)

            returnEdges = []
            for e in tqdm(edges_list, desc="Removing Shards", leave=False):
                if Edge.Length(e) < maximumGap:
                    sv = Edge.StartVertex(e)
                    ev = Edge.EndVertex(e)
                    sEdges = Topology.SuperTopologies(sv, hostTopology, "edge")
                    sn = len(sEdges)
                    eEdges = Topology.SuperTopologies(ev, hostTopology, "edge")
                    en = len(eEdges)
                    # We skip short edges that are "dangling" on a boundary (sn==1 or en==1)
                    # But keep them if sn>=2 AND en>=2
                    if sn >= 2 and en >= 2:
                        returnEdges.append(e)
                else:
                    returnEdges.append(e)
            return returnEdges

        def extendEdges(edges_or_wire, hostTopology, maximumGap=0.5):
            edges_list = force_list_of_edges(edges_or_wire)

            returnEdges = []
            for e in tqdm(edges_list, desc="Extending Edges", leave=False):
                sv = Edge.StartVertex(e)
                ev = Edge.EndVertex(e)
                sEdges = Topology.SuperTopologies(sv, hostTopology, "edge")
                sn = len(sEdges)
                eEdges = Topology.SuperTopologies(ev, hostTopology, "edge")
                en = len(eEdges)
                if sn == 1:
                    ee = Edge.Extend(e, distance=maximumGap, bothSides=False, reverse=True)
                    returnEdges.append(ee)
                elif en == 1:
                    ee = Edge.Extend(e, distance=maximumGap, bothSides=False, reverse=False)
                    returnEdges.append(ee)
                else:
                    returnEdges.append(e)
            return returnEdges

        # ------------------- END HELPER FUNCTIONS ------------------- #

        facesCluster = Cluster.ByTopologies(faces)

        # If externalBoundary has no internal boundaries => fallback
        ibList = Face.InternalBoundaries(externalBoundary)
        if not ibList:
            internalBoundary = externalBoundary
        else:
            internalBoundary = Face.ByWire(ibList[0], tolerance=tolerance)

        # bounding box of that internalBoundary
        bb = Topology.BoundingBox(internalBoundary)
        bb_d = Topology.Dictionary(bb)
        unitU = Dictionary.ValueAtKey(bb_d, 'width') / uSides if uSides else 0
        unitV = Dictionary.ValueAtKey(bb_d, 'length') / vSides if vSides else 0
        uRange = [u * unitU for u in range(uSides)]
        vRange = [v * unitV for v in range(vSides)]

        grid = Grid.EdgesByDistances(face=internalBoundary, uRange=uRange, vRange=vRange, clip=True)
        grid = Topology.Slice(internalBoundary, grid, tolerance=tolerance)
        grid_faces = Topology.Faces(grid)
        skeletons = []

        # For each sub-tile => difference with faces => leftover => skeleton
        for ib in tqdm(grid_faces, desc=f"Processing {len(grid_faces)} tiles", leave=False):
            building_shell = Topology.Slice(ib, facesCluster, tolerance=tolerance)
            wall_faces = Topology.Faces(building_shell)
            walls = []
            for w1 in wall_faces:
                iv = Topology.InternalVertex(w1, tolerance=tolerance)
                # check if iv is inside any original face => skip
                inside_flag = False
                for w2 in faces:
                    if Vertex.IsInternal(iv, w2):
                        inside_flag = True
                        break
                if not inside_flag:
                    # means w1 is "outside" => we skeletonize it
                    walls.append(w1)

            for wall in walls:
                skeleton = Wire.Skeleton(wall, tolerance=0.001)
                skeleton = Topology.Difference(skeleton, facesCluster, tolerance=tolerance)
                # also remove the boundary of that tile => so skeleton lines don't overlap
                # the face boundary
                skeleton = Topology.Difference(skeleton, Face.Wire(wall), tolerance=tolerance)
                skeletons.append(skeleton)

        if not skeletons:
            # No skeleton at all => just return None
            return None

        # Combine skeleton lines + the internal boundary
        skeleton_cluster = Cluster.ByTopologies(skeletons + [internalBoundary])

        # 1) Convert to edges, 2) self-merge, 3) remove shards, 4) possibly extend
        skEdges = Topology.Edges(skeleton_cluster)
        merged_edges = Topology.SelfMerge(Cluster.ByTopologies(skEdges), tolerance=tolerance)
        # now remove shards
        filtered_list = removeShards(merged_edges, skeleton_cluster, maximumGap=maximumGap)
        # then extend edges
        extended_list = extendEdges(filtered_list, skeleton_cluster, maximumGap=maximumGap)

        if not extended_list:
            print("ShellByDisjointFaces - Warning: no edges after extension.")
            return None

        # Now slice the internal boundary with the final skeleton edges
        shell = Topology.Slice(internalBoundary, Cluster.ByTopologies(extended_list), tolerance=tolerance)

        if mergeJunctions:
            # merges vertices that are close together
            vertices = Topology.Vertices(shell)
            # We'll gather "centers" so we unify them
            centers = []
            used = []
            for v in vertices:
                for w in vertices:
                    if not Topology.IsSame(v, w) and (w not in used):
                        if Vertex.Distance(v, w, mantissa=mantissa) < threshold:
                            centers.append(v)
                            used.append(w)

            # Rebuild edges from the shell
            shell_edges = Topology.Edges(shell)  # returns a list
            new_edges = []
            for e in shell_edges:
                sv = Edge.StartVertex(e)
                ev = Edge.EndVertex(e)
                for cV in centers:
                    if Vertex.Distance(sv, cV, mantissa=mantissa) < threshold:
                        sv = cV
                    if Vertex.Distance(ev, cV, mantissa=mantissa) < threshold:
                        ev = cV
                new_edges.append(Edge.ByVertices([sv, ev], tolerance=tolerance))

            cluster = Cluster.ByTopologies(new_edges)
            # Filter x,y coords so we unify
            all_vs = Topology.Vertices(cluster)
            xVals = list(set([Vertex.X(v, mantissa=mantissa) for v in all_vs]))
            yVals = list(set([Vertex.Y(v, mantissa=mantissa) for v in all_vs]))
            xVals.sort()
            yVals.sort()

            xVals = Helper.MergeByThreshold(xVals, 0.5)
            yVals = Helper.MergeByThreshold(yVals, 0.5)

            # Re-check the original edges in the shell, unify coords
            final_edges = []
            for e in shell_edges:
                sv = Edge.StartVertex(e)
                ev = Edge.EndVertex(e)
                svx = Vertex.X(sv, mantissa=mantissa)
                svy = Vertex.Y(sv, mantissa=mantissa)
                evx = Vertex.X(ev, mantissa=mantissa)
                evy = Vertex.Y(ev, mantissa=mantissa)

                # unify X
                for x in xVals:
                    if abs(svx - x) < threshold:
                        svx = x
                        break
                for x in xVals:
                    if abs(evx - x) < threshold:
                        evx = x
                        break
                # unify Y
                for y in yVals:
                    if abs(svy - y) < threshold:
                        svy = y
                        break
                for y in yVals:
                    if abs(evy - y) < threshold:
                        evy = y
                        break

                new_sv = Vertex.ByCoordinates(svx, svy, 0)
                new_ev = Vertex.ByCoordinates(evx, evy, 0)
                final_edges.append(Edge.ByVertices([new_sv, new_ev], tolerance=tolerance))

            cluster2 = Cluster.ByTopologies(final_edges)
            # re-slice the external boundary
            extB = Face.ByWire(Wire.Invert(Face.ExternalBoundary(shell)))
            # or: extB = Face.ByWire(Shell.ExternalBoundary(shell))
            shell = Topology.Slice(extB, cluster2, tolerance=tolerance)

        # Make sure we ended up with a real shell
        if not Topology.IsInstance(shell, "Shell"):
            # attempt a fallback
            try:
                # remove collinear edges from the wires
                temp_wires = [Wire.RemoveCollinearEdges(w, angTolerance=1.0)
                              for w in Topology.Wires(shell)]
                temp_faces = [Face.ByWire(w, tolerance=tolerance)
                              for w in temp_wires]
            except:
                temp_faces = Topology.Faces(shell)
            shell = Shell.ByFaces(temp_faces, tolerance=tolerance)

        # Transfer dictionaries if desired
        if transferDictionaries:
            selectors = []
            for f in faces:
                d = Topology.Dictionary(f)
                # place a vertex in the face
                s = Topology.InternalVertex(f, tolerance=tolerance)
                s = Topology.SetDictionary(s, d)
                selectors.append(s)
            shell = Topology.TransferDictionariesBySelectors(
                topology=shell,
                selectors=selectors,
                tranFaces=True,
                tolerance=tolerance
            )

        return shell

In [11]:
def build_skeleton_with_disjoint_faces(
        externalBoundary,
        subFaces,
        maximumGap=0.5,
        mergeJunctions=False,
        threshold=0.5,
        uSides=1,
        vSides=1,
        tolerance=0.0001
):
    """
    Calls the Shell.ByDisjointFaces function on the 
    externalBoundary + subFaces to produce a skeleton Shell.
    """
    skeleton = Shell2.ByDisjointFaces(
        externalBoundary=externalBoundary,
        faces=subFaces,
        maximumGap=maximumGap,
        mergeJunctions=mergeJunctions,
        threshold=threshold,
        uSides=uSides,
        vSides=vSides,
        transferDictionaries=False,
        mantissa=6,
        tolerance=tolerance
    )
    return skeleton


def extrude_skeleton_into_floors(
        skeletonShell,
        floor_height=3.0,
        floors=3
):
    """
    Given a 2D skeleton shell, extrude it upward into N floors
    (like 'stacking' the 2D skeleton in Z direction).

    Returns a cluster of extruded shells (or faces).
    """
    extruded_list = []

    for level in range(floors):
        # shift in Z
        shift_z = level * floor_height
        moved = Topology.Translate(skeletonShell, 0, 0, shift_z)
        extruded_list.append(moved)

    return Cluster.ByTopologies(extruded_list)

In [12]:
my_skeleton = build_skeleton_with_disjoint_faces(
    externalBoundary=shell,
    subFaces=faces,
    maximumGap=0.5,
    mergeJunctions=True,
    threshold=0.5,
    uSides=2,
    vSides=2
)

Processing 4 tiles:   0%|          | 0/4 [00:00<?, ?it/s]

Removing Shards:   0%|          | 0/70 [00:00<?, ?it/s]

Extending Edges:   0%|          | 0/30 [00:00<?, ?it/s]

Edge.ByStartVertexEndVertex - Error: The input vertexA and vertexB parameters are the same vertex. Returning None.
Edge.ByVertices - Error: Could not create an edge. Returning None.
caller name: ByDisjointFaces
Edge.ByStartVertexEndVertex - Error: The input vertexA and vertexB parameters are the same vertex. Returning None.
Edge.ByVertices - Error: Could not create an edge. Returning None.
caller name: ByDisjointFaces
Edge.ByStartVertexEndVertex - Error: The input vertexA and vertexB parameters are the same vertex. Returning None.
Edge.ByVertices - Error: Could not create an edge. Returning None.
caller name: ByDisjointFaces
Edge.ByStartVertexEndVertex - Error: The input vertexA and vertexB parameters are the same vertex. Returning None.
Edge.ByVertices - Error: Could not create an edge. Returning None.
caller name: ByDisjointFaces
Edge.ByStartVertexEndVertex - Error: The input vertexA and vertexB parameters are the same vertex. Returning None.
Edge.ByVertices - Error: Could not create

In [13]:
print(my_skeleton)

In [14]:
print(f"floors_cluster object: {my_skeleton}")

floors_cluster object: <topologic_core.Shell object at 0x128b25bf0>


In [15]:
faces = Topology.Faces(my_skeleton)
print(f"Number of faces: {len(faces)}")

Number of faces: 7


In [16]:
edges = Topology.Edges(my_skeleton)
print(f"Number of edges: {len(edges)}")

Number of edges: 34


In [17]:
Topology.Show(my_skeleton, renderer=renderer)